In [112]:
!pip install pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
spark = SparkSession.builder.appName('spark-intro').getOrCreate()
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable 
import numpy as np
import seaborn as sns
import pandas as pd
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import feature,regression, evaluation,Pipeline,PipelineModel,classification
from pyspark.ml.evaluation import RegressionEvaluator,MulticlassClassificationEvaluator
from pyspark.sql.functions import when
from pyspark.sql import functions as fn, Row
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
from google.colab import drive
drive.mount('/content/drive')
WatchData = spark.read.csv('/content/drive/MyDrive/Activity recognition exp/Watch_Data.csv',header=True, inferSchema= True)
print('Watch Data shape: ',(WatchData.count(), len(WatchData.columns)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AnalysisException: ignored

In [111]:
from pyspark.sql.functions import *
import time
from pyspark.sql import functions as f
from pyspark.sql import types as t

WatchData2 = WatchData.withColumn("Arrival_Time",f.to_timestamp(WatchData['Arrival_Time']/1000))
WatchData2 = WatchData2.withColumn("Creation_Time",f.to_timestamp(WatchData['Creation_Time']/1000))

WatchData3 = WatchData2.withColumn('Arrival_Date', split(col('Arrival_Time'),' ').getItem(0))
WatchData3 = WatchData3.withColumn('Arrival_time', split(col('Arrival_Time'),' ').getItem(1))

WatchData3 = WatchData3.withColumn('Creation_Date', split(col('Creation_Time'),' ').getItem(0))
WatchData3 = WatchData3.withColumn('Creation_time', split(col('Creation_Time'),' ').getItem(1))

WatchData3.show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [108]:
WatchData_display = WatchData3[['Arrival_time','Creation_time','User','Model','bjerkmean','Sensor','x','y','z','gt']]
WatchData_display.summary().show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [105]:
from pyspark.sql.functions import col,isnan, when, count
WatchData_display.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in WatchData_display.columns]).show()

+------------+-------------+----+-----+---------+------+---+---+---+---+
|Arrival_Time|Creation_Time|User|Model|bjerkmean|Sensor|  x|  y|  z| gt|
+------------+-------------+----+-----+---------+------+---+---+---+---+
|           0|            0|   0|    0|        0|     0|  0|  0|  0|  0|
+------------+-------------+----+-----+---------+------+---+---+---+---+



In [106]:
WatchData_display.select('User').distinct().show()

+----+
|User|
+----+
|   g|
|   f|
|   e|
|   h|
|   d|
|   c|
|   i|
|   b|
|   a|
+----+



In [107]:
WatchData_display.filter(WatchData_display.User == "g").show()

+------------+-------------+----+-------+-----------+------+-----------+------------+------------+----------+
|Arrival_Time|Creation_Time|User|  Model|  bjerkmean|Sensor|          x|           y|           z|        gt|
+------------+-------------+----+-------+-----------+------+-----------+------------+------------+----------+
|  1.42469E12|   1.15999E14|   g|lgwatch|0.052289781|   GYR|  0.4643097| -0.27870178|  0.29130554|      bike|
|  1.42469E12|   2.58174E13|   g|   gear| 0.50752904|   Acc| -10.836756|  -8.8567505| -0.39863458|stairsdown|
|  1.42469E12|   2.59805E13|   g|   gear|0.004761482|   Acc| -6.4727244|    4.981735|    6.487688|      bike|
|  1.42469E12|   1.16106E14|   g|lgwatch|0.668245788|   Acc|  -9.077728|   5.3290253|  -5.7785645|      bike|
|  1.42469E12|   1.16075E14|   g|lgwatch|0.830916656|   Acc| -4.3693542|    7.066696|   -5.500061|      bike|
|  1.42469E12|    1.1585E14|   g|lgwatch|0.884905506|   Acc|  -6.023712|   5.0624237|   -4.812134|      bike|
|  1.42469